In [ ]:
# Testing deconvolutions for temp test folder (containing 1 image)

from nrrd import read, write
from tifffile import imwrite
import shutil, os
import time

# delete old directory
try:
    shutil.rmtree(r"E:\TMP\Keivan\de\temp")
    os.remove(r"E:\TMP\Keivan\de\y00010920.x00005472.nrrd")
except FileNotFoundError:
    print("File not found")
    
print("Old directory deleted")
time.sleep(2)

# run fnt_cube_processor.py --no-destripe --wavelength_ex 642 --wavelength_em 690    <- TODO: program crashes
! python .\fnt_cube_processor.py -i E:\TMP\Keivan\or -o E:\TMP\Keivan\de -n 1 -d --no-destripe --nimm 1.42 --dxy 0.7 --dz 1.2
# time.sleep(2)
# img, header = read(r"E:\TMP\Keivan\de\y00010920.x00005472.nrrd")
# imwrite(r"E:\TMP\Keivan\de\test_gd_de_1.42_600.tif", img, compression=1)

# img, header = read(r"E:\TMP\Keivan\or\y00010920.x00005472.nrrd")
# imwrite(r"E:\TMP\Keivan\de\test_or.tif", img, compression=1)

In [2]:
# Testing deconvolutions for temp test folder (containing 1 image)

from nrrd import read, write
from tifffile import imwrite
import shutil, os
import time

# delete old directory
try:
    shutil.rmtree(r"E:\TMP\Aidan\out_test_single\temp")
    os.remove(r"E:\TMP\Aidan\out_test_single\y00006400.x00019200.nrrd")
except FileNotFoundError:
    print("File not found")
    
print("Old directory deleted")
time.sleep(0.5)

# run fnt_cube_processor.py
! python .\fnt_cube_processor.py -i E:\TMP\Aidan\tmp -o E:\TMP\Aidan\out_test_single -n 1 -d  
# time.sleep(2)
img, header = read(r"E:\TMP\Aidan\out_test_single\y00006400.x00019200.nrrd")
imwrite(r"E:\TMP\Aidan\out_test_single\test_decon.tif", img, compression=1)

Old directory deleted


C:\Users\ajan\AppData\Local\anaconda3\envs\stitching\Lib\site-packages\pycudadecon\_libwrap.py:44: UserWarning: Unable to find function: camcor_interface_init in libcudaDecon
  @lib.function
C:\Users\ajan\AppData\Local\anaconda3\envs\stitching\Lib\site-packages\pycudadecon\_libwrap.py:51: UserWarning: Unable to find function: camcor_interface in libcudaDecon
  @lib.function

FNT Cube Processor: 100%|##########| 1/1 [00:05<00:00,  5.73s/ cubes]


Two objects are distinguishable in xy-plane if they are 801 nm apart. The camera pixel size is 700 nm.
Two objects are distinguishable in z-axis if they are 8662 nm apart. The z-step is 1400 nm.
full width half maxima of xy-plane is 958.9 nm.
full width half maxima of z-axis is 9202.4 nm.
otf shape:  (0, 8, 18)
image size before pad:  (320, 320, 160)
image size after pad:  (324, 336, 180)
image size after first decon:  (324, 336, 180)
image size after gaussian:  (324, 336, 180)
image size after second decon:  (324, 336, 180)
image size after crop:  (320, 320, 160)


In [ ]:
def get_next_good_dim(n: int):
    print("called get_next_good_dim")
    while True:
        temp = n
        for prime in [2, 3, 5, 7]:
            while temp % prime == 0:
                temp //= prime
        if temp != 1:
            n += 1
        else:
            break
    return n

get_next_good_dim(160)

In [ ]:
def convert_file(input_nrrd):
    img, header = read(input_nrrd.__str__())
    tif_path = Path(output_tif_folder) / (input_nrrd.name[:-4] + 'tif')
    imwrite(tif_path.__str__(), img, compression=1)
    return 1

In [ ]:
# Testing deconvolutions for real folder (this one takes a long time)
from nrrd import read
from tifffile import imwrite
from pathlib import Path
from os import system 
from multiprocessing import Pool


# running fnt_cube_processor with 8 threads
input_folder = r"E:\TMP\Aidan\z00003200"
output_nrrd_folder = r"E:\TMP\Aidan\z00003200_deconv_nrrd"
output_tif_folder = r"E:\TMP\Aidan\z00003200_tif"
num_threads = 8

system(fr"python .\fnt_cube_processor.py -i {input_folder} -o {output_nrrd_folder} -n {num_threads} -d")

# convert to tif files
nrrd_paths = [f for f in Path(output_nrrd_folder).iterdir() if f.is_file()]

# convert single file to tif
def convert_file(input_nrrd: Path):
    img, header = read(input_nrrd.__str__())
    tif_path = Path(output_tif_folder) / (input_nrrd.name[:-4] + 'tif')
    imwrite(tif_path.__str__(), img, compression=1)
    return 1

for path in nrrd_paths:
    convert_file(path)

print("All nrrd -> tif conversions completed successfully")


In [1]:
!python convert.py -i E:\Aidan\AD6-training-split -o E:\Aidan\ims_output\split.ims -dx 1.8 -dy 1.8 -dz 1.8

2025-02-04 08:41:03: converting AD6-training-split to ims ... 
	tiff to ims conversion command:
		 imaris\ImarisConvertiv.exe --input E:\Aidan\AD6-training-split\img_000000.tif --output E:\Aidan\ims_output\split.ims --inputformat TiffSeries --nthreads 48 --compression 3 --voxelsize 1.80-1.80-1.80 --logprogress


Following command succeeded:
	 imaris\ImarisConvertiv.exe --input E:\Aidan\AD6-training-split\img_000000.tif --output E:\Aidan\ims_output\split.ims --inputformat TiffSeries --nthreads 48 --compression 3 --voxelsize 1.80-1.80-1.80 --logprogress



IMS:   0%|          | 0/100 [00:00<?, ? %/s]
IMS:   0%|          | 0.0/100 [00:07<?, ? %/s]
IMS:   0%|          | 0.1/100 [00:08<04:07,  2.48s/ %]
IMS:   0%|          | 0.3/100 [00:08<02:45,  1.66s/ %]
IMS:   0%|          | 0.4/100 [00:08<03:06,  1.87s/ %]
IMS:   0%|          | 0.5/100 [00:08<03:19,  2.00s/ %]
IMS:   1%|          | 0.7/100 [00:09<02:56,  1.78s/ %]
IMS:   1%|          | 0.8/100 [00:09<03:06,  1.88s/ %]
IMS:   1%|          | 0.9/100 [00:09<03:12,  1.95s/ %]
IMS:   1%|1         | 1.1/100 [00:09<02:59,  1.82s/ %]
IMS:   1%|1         | 1.2/100 [00:10<03:05,  1.88s/ %]
IMS:   1%|1         | 1.3/100 [00:10<03:10,  1.93s/ %]
IMS:   2%|1         | 1.8/100 [00:21<13:16,  8.11s/ %]
IMS:   2%|2         | 2.3/100 [00:22<10:24,  6.39s/ %]
IMS:   3%|2         | 2.7/100 [00:22<08:54,  5.50s/ %]
IMS:   3%|3         | 3.0/100 [00:22<08:05,  5.01s/ %]
IMS:   3%|3         | 3.1/100 [00:31<12:51,  7.96s/ %]
IMS:   4%|3         | 3.7/100 [00:32<10:42,  6.67s/ %]
IMS:   4%|4         | 4.2/1